In [1]:
# For usage on google colab:
!git clone https://github.com/ahans30/Binoculars.git
%cd Binoculars
!pip install -e .

In [2]:
from binoculars import Binoculars

ModuleNotFoundError: No module named 'binoculars'

In [1]:
import numpy as np
import torch
import transformers
import pickle as pkl
import pandas as pd
from google.colab import files

/opt/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
bino = Binoculars()

In [3]:
!ls

encodings.pkl        ob_logits.pkl        process_logits.ipynb
extract_logits.ipynb pad_token_id.pkl     raid_test_1000.csv
generate_walks.ipynb pf_logits.pkl        raid_test_5000.csv


In [6]:
df = pd.read_csv("raid_test_5000.csv")

In [7]:
df.shape

(5000, 13)